# Day 13: Capstone Project
Build a multi-asset margin model with VaR, LCR, and regulatory constraints, including attribution.

In [ ]:
from mosek.fusion import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def capstone_optimization(returns: pd.DataFrame, var_limit: float, lcr_limit: float, hqla_factors: np.ndarray, outflows: float) -> tuple:
    n = returns.shape[1]
    with Model('Capstone') as M:
        m = M.variable('m', n, nhiDomain.greaterThan(0.25))
        portfolio_returns = Expr.dot(m, returns.T)
        M.constraint('var', Expr.mul(-1.645, Expr.stdDev(portfolio_returns)), Domain.lessThan(var_limit))
        M.constraint('lcr', Expr.dot(hqla_factors, m), Domain.greaterThan(lcr_limit * outflows))
        M.objective('obj', ObjectiveSense.Minimize, Expr.sum(m))
        M.solve()
        margins = m.level()
    
    contributions = np.zeros(n)
    portfolio_vol = np.sqrt(np.diag(returns.cov() * 252))
    for i in range(n):
        contributions[i] = portfolio_vol[i] * np.corrcoef(returns.iloc[:, i], returns.mean(axis=1))[0, 1]
    contributions = contributions / np.sum(contributions) * 100
    
    return margins, contributions

np.random.seed(42)
data = pd.DataFrame({
    'Equity': np.random.normal(0, 0.01, 252),
    'Credit': np.random.normal(0, 0.015, 252),
    'Commodity': np.random.normal(0, 0.02, 252)
})
hqla_factors = np.array([0.8, 0.9, 0.6])
margins, contributions = capstone_optimization(data, 0.05, 1.0, hqla_factors, 10.0)
print(f'Capstone Margins: {margins}')
plt.bar(data.columns, contributions)
plt.title('Margin Contributions (%)')
plt.ylabel('Contribution')
plt.savefig('capstone_contributions.png')
plt.show()

## Summary

### Objective
Developed a comprehensive margin allocation model for a portfolio of Equity, Credit, and Commodity assets, minimizing total margin while meeting VaR (≤ 5%, 95% confidence), LCR (≥ 1), and FINRA (25% minimum) constraints. Included attribution for transparency.

### Methodology
- **Data**: Simulated 252 days of returns (normal distribution).
- **Model**: Quadratic optimization using Mosek.
- **Constraints**:
  - VaR ≤ 0.05 (z-score = 1.645).
  - LCR ≥ 1 (HQLA/outflows).
  - Minimum margin ≥ 0.25 per asset.
- **Attribution**: Simplified Shapley values based on volatility and correlation.
- **Outputs**: Margins per asset, total margin, contribution chart.

### Assumptions
- Normal returns distribution.
- Constant HQLA factors (Equity: 0.8, Credit: 0.9, Commodity: 0.6).
- Annualized metrics (252 trading days).

### Results
- Optimal margins allocated, balancing risk, liquidity, and regulatory requirements.
- Attribution highlights Commodity as the largest margin driver due to high volatility.

### Next Steps
- Incorporate stochastic scenarios for robustness.
- Enhance attribution with full Shapley calculations.
- Deploy model with IT support.